<a href="https://colab.research.google.com/github/Guhan2348519/SPR_labs/blob/main/2348519_SPR_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install SpeechRecognition pydub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 33.4 MB/s eta 0:00:00


In [18]:
from google.colab import files
uploaded = files.upload()


Saving Harvard list 01.wav to Harvard list 01.wav


In [19]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def transcribe_audio_file(path):
    # Initialize recognizer
    recognizer = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    # Split the audio into chunks based on silence
    chunks = split_on_silence(
        sound,
        min_silence_len=700,
        silence_thresh=sound.dBFS-14,
        keep_silence=500
    )

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""

    # Process each chunk
    for i, chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        chunk.export(chunk_filename, format="wav")

        with sr.AudioFile(chunk_filename) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data)
            except sr.UnknownValueError:
                print("Could not understand audio in chunk", i)
            else:
                whole_text += f"{text.capitalize()}. "

    return whole_text

# Path to uploaded audio file (change 'your_audio.wav' to your uploaded file name)
uploaded_file = list(uploaded.keys())[0]
transcription = transcribe_audio_file(uploaded_file)

print("Transcription:\n", transcription)


Transcription:
 Harvard list number one. The birch canoe slid on the smooth planks. Glue the sheet to the dark blue background. It's easy to tell the depth of a well. These days a chicken leg is a rare dish. Rice is often served in round bowls. The juice of lemons makes fine punch. The box was thrown beside the parked truck. The hawks were fed chopped corn and garbage. 4 hours of steady work faces. Large size in stockings is hard to sell. 


In [11]:
!apt-get install -y portaudio19-dev
!pip install sounddevice


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (267 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [7]:
!pip install sounddevice scipy
